<a href="https://colab.research.google.com/github/rajagopalmotivate/AIPhysics/blob/master/SEM_try1c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paper https://www.nature.com/articles/sdata2018172

idea 1: auto ml version of sem 

idea 2 : ensemble of inception, vgg, resnet based approach 

idea 3: denoising AE 

idea 4: inpainting to uncover hidden things 



In [0]:
#delete the directory named mynewfiles including its subfolders with the rm -r command 
!rm -r mynewfiles

In [0]:
!mkdir mynewfiles

In [3]:
# source URL for dataset: https://b2share.eudat.eu/records/72758204db9044ab8b3e6b6c4d2eb576

!wget https://b2share.eudat.eu/api/files/87275d32-0908-45bc-89e2-8f3d86029283/Porous_Sponge.tar -O /tmp/dir1.tar

--2019-08-29 18:29:51--  https://b2share.eudat.eu/api/files/87275d32-0908-45bc-89e2-8f3d86029283/Porous_Sponge.tar
Resolving b2share.eudat.eu (b2share.eudat.eu)... 86.50.166.74
Connecting to b2share.eudat.eu (b2share.eudat.eu)|86.50.166.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123289600 (118M) [application/octet-stream]
Saving to: ‘/tmp/dir1.tar’

/tmp/dir1.tar       100%[===================>] 117.58M  10.3MB/s    in 13s     

2019-08-29 18:30:06 (8.79 MB/s) - ‘/tmp/dir1.tar’ saved [123289600/123289600]



In [4]:
!ls /tmp


dir1.tar  inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
dir2.tar  pymp-s3da6pxq
dir3.tar


In [0]:
!ls mynewfiles

In [6]:
!tar xvf  /tmp/dir1.tar -C mynewfiles

Porous_Sponge/
Porous_Sponge/L0_cf9f11694caad1e286bedf582bb45456.jpg
Porous_Sponge/L0_7333ded956b39296bbf8379c39d70682.jpg
Porous_Sponge/L0_35da4528991365232bec1278d5456bbc.jpg
Porous_Sponge/L0_fc3543ca184b0e8340559b6ed6a5686a.jpg
Porous_Sponge/L0_9df743ce3d6097d610fd145c07895ea0.jpg
Porous_Sponge/L0_1bb457a9dae682696fdce19955bc1360.jpg
Porous_Sponge/L0_18d2c6b2f552be88041cba8beb378ab4.jpg
Porous_Sponge/L0_1c34778dec5a1701d689a7067c4813eb.jpg
Porous_Sponge/L0_35fb9e4f21c0f68a9130df0746d221d5.jpg
Porous_Sponge/L0_0f33d3d0e9efbe60c16a2aa1af3e128d.jpg
Porous_Sponge/L0_df0c51c3537b6933ea08467339d3eaee.jpg
Porous_Sponge/L0_273b2fe2c224173a255d9c03d4c98c32.jpg
Porous_Sponge/L0_2ce557ad129572b8d8258f440fbdec2e.jpg
Porous_Sponge/L0_74f696205ed9a1c0a355d0a7fa59addf.jpg
Porous_Sponge/L0_f38f03e44dd36406590945a374402a26.jpg
Porous_Sponge/L0_6fcbe32aaa0e3bcf95d61460991caeb6.jpg
Porous_Sponge/L0_109dd70cc8f44651f0e1f76837057e56.jpg
Porous_Sponge/L0_028965453585464cfda6b7df06e0bc39.jpg
Porous_Sponge

In [7]:
!ls mynewfiles

Porous_Sponge


In [8]:
!wget https://b2share.eudat.eu/api/files/87275d32-0908-45bc-89e2-8f3d86029283/Films_Coated_Surface.tar -O /tmp/dir2.tar

KeyboardInterrupt: ignored

In [0]:
!ls /tmp


In [0]:
!tar xvf  /tmp/dir2.tar -C mynewfiles

In [0]:
!ls mynewfiles

In [0]:
!wget https://b2share.eudat.eu/api/files/87275d32-0908-45bc-89e2-8f3d86029283/Fibres.tar -O /tmp/dir3.tar

In [0]:
!tar xvf  /tmp/dir3.tar -C mynewfiles

In [0]:
!ls mynewfiles

#Goal: Just understand the high level concept 
###( let's understand the big picture , don't worry about the coding details right now..)

In [0]:
!ls mynewfiles

In [0]:
!pwd

In [0]:
!ls

In [0]:
import os

rock_dir = os.path.join('mynewfiles/Fibres')
paper_dir = os.path.join('mynewfiles/Films_Coated_Surface')
scissors_dir = os.path.join('mynewfiles/Porous_Sponge')

print('total training rock images:', len(os.listdir(rock_dir)))
print('total training paper images:', len(os.listdir(paper_dir)))
print('total training scissors images:', len(os.listdir(scissors_dir)))

rock_files = os.listdir(rock_dir)
print(rock_files[:10])

paper_files = os.listdir(paper_dir)
print(paper_files[:10])

scissors_files = os.listdir(scissors_dir)
print(scissors_files[:10])

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2

next_rock = [os.path.join(rock_dir, fname) 
                for fname in rock_files[pic_index-2:pic_index]]
next_paper = [os.path.join(paper_dir, fname) 
                for fname in paper_files[pic_index-2:pic_index]]
next_scissors = [os.path.join(scissors_dir, fname) 
                for fname in scissors_files[pic_index-2:pic_index]]

for i, img_path in enumerate(next_rock+next_paper+next_scissors):
  #print(img_path)
  img = mpimg.imread(img_path)
  print(img_path)
  print(img.shape)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

In [0]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "mynewfiles/"

datagen = ImageDataGenerator(validation_split=0.1, rescale=1./255,      rotation_range=40,
     width_shift_range=0.2,
     height_shift_range=0.2,
     shear_range=0.2,
     zoom_range=0.2,
     horizontal_flip=True,
     fill_mode='nearest')

train_generator = datagen.flow_from_directory(
    TRAINING_DIR, 
    subset='training',
    target_size=(150,150),
         shuffle=True, batch_size = 2, 
	  class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    TRAINING_DIR,
    subset='validation',
    	target_size=(150,150),
         shuffle=True, batch_size = 2, 
	class_mode='categorical'
)




test_datagen = ImageDataGenerator( rescale=1./255)


test_generator = test_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150,150),
    batch_size=1,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

In [0]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [0]:
last_layer = pre_trained_model.get_layer('mixed7')
print(last_layer.output_shape)
last_output = last_layer.output

In [0]:
ispostmixed = False 

for layer in pre_trained_model.layers:
  if layer.name == "mixed6":
    ispostmixed = True
  if ispostmixed == True:
    layer.trainable = True
    print(layer.name)

In [0]:
from tensorflow.keras import layers

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
x = layers.BatchNormalization() (x)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(5000, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
x = layers.BatchNormalization() (x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.BatchNormalization() (x)
x = layers.Dense(3, activation='softmax')(x)





In [0]:
from tensorflow.keras import Model
model = Model(pre_trained_model.input, x)


In [0]:

modelOLD = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])


In [0]:
model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [0]:
for alayer in model.layers:
  print(str(alayer.name) + '    \t \t \t ' +  str(alayer.trainable))

In [0]:
history = model.fit_generator(train_generator, epochs=4, validation_data = validation_generator , verbose = 1)

#model.save("rps.h5")

In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [0]:
test_generator.reset()


In [0]:
#STEP_SIZE_TEST=validation_generator.n//validation_generator.batch_size

import numpy as np

predictions =model.predict_generator(test_generator, steps=100, verbose=1)
print(predictions)

predicted_class_indices=np.argmax(predictions,axis=1)
print(predicted_class_indices)

labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictionsanswers = [labels[k] for k in predicted_class_indices]

print(predictionsanswers)


In [0]:
filenames=test_generator.filenames

print(filenames)

In [0]:
for i in range( 1):
 print(predictionsanswers[i] , filenames[i] ) 



In [0]:
test_generator.reset()
x , y = next(test_generator)

print(x.shape)

In [0]:
predictions =model.predict_generator(test_generator, steps=100, verbose=1)
print(predictions)

predicted_class_indices=np.argmax(predictions,axis=1)
print(predicted_class_indices)

labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictionsanswers = [labels[k] for k in predicted_class_indices]

print(predictionsanswers)


In [0]:

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)

In [0]:
def predictyforaimage(filepath):
  img = image.load_img(filepath, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  predictions = model.predict(images, batch_size=1)
  print (predictions)




In [0]:
filepath1 = "mynewfiles/Fibres/L9_ee1dc9c124191a5bd7ffc9c2f3e23551.jpg"

predictyforaimage(filepath1)

In [0]:
filepath1 = "mynewfiles/Porous_Sponge/L0_bbaf2aba3f987b75e124f6c06eb6e84a.jpg"
predictyforaimage(filepath1)


In [0]:
filepath1 = "mynewfiles/Fibres/L9_3f42fc660eb039ea7caaddc54fe2b549.jpg"
predictyforaimage(filepath1)



In [0]:

filepath1 = "mynewfiles/Porous_Sponge/L0_ae1c8f14a5ade3d92540e283da87d528.jpg"
predictyforaimage(filepath1)

In [0]:
filepath2 = "mynewfiles/Films_Coated_Surface/L3_ed7409b9ab4b6c78a57925bea608596d.jpg"
predictyforaimage(filepath2)